**Import the libraries**

In [5]:
import cv2
import dlib
import face_recognition  # <-- Add this import
import os
import numpy as np

**The required files for using this code**

1. deploy.prototxt "https://github.com/opencv/opencv/blob/master/samples/dnn/face_detector/deploy.prototxt"
2. res10_300x300_ssd_iter_140000.caffemodel "https://github.com/keyurr2/face-detection/blob/master/res10_300x300_ssd_iter_140000.caffemodel"

In [10]:
# Define your name
YOUR_NAME = "king"  # Change this to your actual name

# Provide correct paths to model files
MODEL_PATH = r"C:\Users\sfazeli\Downloads\Face tracker"  # Change this if needed
config_file = os.path.join(MODEL_PATH, "deploy.prototxt")
model_file = os.path.join(MODEL_PATH, "res10_300x300_ssd_iter_140000.caffemodel")

# Load deep learning model for face detection
net = cv2.dnn.readNetFromCaffe(config_file, model_file)

# Initialize dlib tracker
tracker = dlib.correlation_tracker()
trackingFace = False  # Whether we are tracking your face
savedFaceEncoding = None  # Store your face encoding

# Initialize webcam
capture = cv2.VideoCapture(0)

if not capture.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = capture.read()
    if not ret:
        break

    # Convert frame to blob for deep learning model
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))

    if not trackingFace:
        # Detect faces using deep learning
        net.setInput(blob)
        detections = net.forward()

        maxConfidence = 0
        yourFace = None

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                box = detections[0, 0, i, 3:7] * [w, h, w, h]
                x, y, x2, y2 = box.astype("int")

                # Track the face with the highest confidence (assuming it's yours)
                if confidence > maxConfidence:
                    yourFace = (x, y, x2, y2)
                    maxConfidence = confidence

        # If a face is detected, initialize tracker
        if yourFace:
            x, y, x2, y2 = yourFace
            tracker.start_track(frame, dlib.rectangle(x, y, x2, y2))
            trackingFace = True

    else:
        # Update tracker
        tracker.update(frame)
        pos = tracker.get_position()

        # Get coordinates of tracked face
        x, y, x2, y2 = int(pos.left()), int(pos.top()), int(pos.right()), int(pos.bottom())

        # Draw tracking box
        cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)

        # Display confidence score and name
        text = f"{YOUR_NAME} ({maxConfidence*100:.2f}%)"
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display output
    cv2.imshow("AI-Powered Face Tracker", frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
capture.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

**Individually for your face**

In [ ]:
# Define your name
YOUR_NAME = "King"  # This variable stores your name, which will be displayed when your face is detected.

# Provide correct paths to model files
MODEL_PATH = r"C:\Users\sfazeli\Downloads\Face tracker"  # Specifies the folder where the face detection model files are stored.
config_file = os.path.join(MODEL_PATH, "deploy.prototxt") # Specifies the path to the prototxt file.
model_file = os.path.join(MODEL_PATH, "res10_300x300_ssd_iter_140000.caffemodel") # Specifies the path to the caffemodel file.

# Load deep learning model for face detection
net = cv2.dnn.readNetFromCaffe(config_file, model_file) # Loads the pre-trained deep learning model for face detection using OpenCV's dnn module.

# Initialize dlib tracker
tracker = dlib.correlation_tracker() # dlib.correlation_tracker() is used to follow your face after detection
trackingFace = False  #  Boolean flag to check if tracking is active.
savedFaceEncoding = None  # Stores the unique encoding of your face.
savedBoundingBox = None  # Store your bounding box. Stores the coordinates of the tracked face.

# Initialize webcam
capture = cv2.VideoCapture(0) # ✔ Opens the webcam to start capturing video frames.

if not capture.isOpened(): # ✔ Checks if the webcam is opened successfully.
    print("Error: Could not open webcam.") # ✔ Prints an error message if the webcam fails to open.
    exit()

while True: # ✔ Starts an infinite loop to capture video frames from the webcam.
    ret, frame = capture.read()
    if not ret:
        break

    # Convert frame to blob for deep learning model
    h, w = frame.shape[:2] # ✔ Extracts the height and width of the frame.
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0)) # ✔ Converts the frame to a blob format for the deep learning model.Normalizes pixel values by subtracting (104.0, 177.0, 123.0) (mean subtraction).

    if not trackingFace:
        # Detect faces using deep learning
        net.setInput(blob) #Sends the processed image (blob) to the deep learning model.
        detections = net.forward() # Detects faces in the frame using the deep learning model.

        maxConfidence = 0
        yourFace = None # Stores the coordinates of your face.
        bestEncoding = None # Stores the encoding of the best match.

        for i in range(detections.shape[2]): # Iterates over the detected faces.
            confidence = detections[0, 0, i, 2] # Extracts the confidence score of the detected face.
            if confidence > 0.7:  # Confidence threshold increased for better accuracy
                box = detections[0, 0, i, 3:7] * [w, h, w, h] # Extracts the bounding box coordinates of the detected face.
                x, y, x2, y2 = box.astype("int") # Converts the coordinates to integers.

                # Extract the face ROI and compute encoding
                faceROI = frame[y:y2, x:x2] # Extracts the face region of interest (ROI) from the frame.
                
                if faceROI.shape[0] > 0 and faceROI.shape[1] > 0: # Checks if the face ROI is valid.
                    rgbFace = cv2.cvtColor(faceROI, cv2.COLOR_BGR2RGB) # Converts the face ROI to RGB format.
                    encodings = face_recognition.face_encodings(rgbFace) # Computes the face encoding using face_recognition library.

                    if encodings:
                        faceEncoding = encodings[0]  # Take the first detected face

                        # If this is the first time, store your face encoding
                        if savedFaceEncoding is None:
                            savedFaceEncoding = faceEncoding
                            savedBoundingBox = (x, y, x2, y2)
                            print(f"Face of {YOUR_NAME} saved for tracking!")

                        # Compare this face with your stored face
                        match = face_recognition.compare_faces([savedFaceEncoding], faceEncoding)[0]

                        if match and confidence > maxConfidence:
                            yourFace = (x, y, x2, y2)
                            maxConfidence = confidence
                            bestEncoding = faceEncoding # Store the best encoding

        # If a matching face is detected, initialize tracker
        if yourFace:
            x, y, x2, y2 = yourFace
            tracker.start_track(frame, dlib.rectangle(x, y, x2, y2)) # Initializes the dlib tracker with the bounding box of the detected face.
            trackingFace = True # Sets the trackingFace flag to True to indicate that tracking is active.
            savedBoundingBox = (x, y, x2, y2)

    else:
        # Update tracker
        tracker.update(frame) # Updates the tracker with the new frame.
        pos = tracker.get_position() # Gets the position of the tracked face.

        # Get coordinates of tracked face
        x, y, x2, y2 = int(pos.left()), int(pos.top()), int(pos.right()), int(pos.bottom())

        # If tracking drifts too much, reset tracking
        if abs(x - savedBoundingBox[0]) > 50 or abs(y - savedBoundingBox[1]) > 50:
            trackingFace = False
            continue

        # Draw tracking box
        cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)

        # Display confidence score and name
        text = f"{YOUR_NAME} ({maxConfidence*100:.2f}%)"
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2) # Displays the name and confidence score of the detected face.

    # Display output
    cv2.imshow("AI-Powered Face Tracker", frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
capture.release()
cv2.destroyAllWindows()


Face of King saved for tracking!


KeyboardInterrupt: 